In [1]:
import numpy as np
import librosa
import moviepy.editor as mp
import os
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('modified_df.csv')
df

,Emotion,Dialogue_ID,Utterance_ID,Sentiment,0,1,2,3,4,5,...,118,119,120,121,122,123,124,125,126,127
0,neutral,0,0,neutral,-40.785557,-31.903383,-28.509054,-24.394083,-23.567312,-28.972204,...,-57.298710,-56.888195,-57.355335,-57.385845,-58.302578,-59.111305,-59.915573,-61.931507,-67.367920,-75.377335
1,neutral,0,1,neutral,-46.091118,-41.264591,-34.173096,-27.214109,-26.786467,-31.770670,...,-62.016983,-63.054230,-63.258858,-62.921265,-64.128593,-64.542862,-64.874359,-65.916077,-69.985847,-76.369377
2,neutral,0,2,neutral,-49.231205,-42.710953,-39.304115,-34.735527,-34.517563,-38.112541,...,-64.425728,-64.667534,-65.593277,-65.887383,-66.269699,-67.071754,-67.932968,-69.560272,-74.179443,-79.334343
3,neutral,0,3,neutral,-45.178123,-41.165024,-38.436745,-33.843090,-30.376688,-31.385689,...,-60.421524,-60.690266,-60.210945,-59.100220,-59.930977,-60.668087,-62.051445,-62.430584,-67.484306,-75.673409
4,surprise,0,4,positive,-48.915817,-44.924217,-41.524128,-35.188869,-31.114756,-31.212828,...,-61.329353,-61.320774,-61.754005,-63.535156,-64.087585,-64.762703,-66.426491,-68.073280,-72.419991,-79.377052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,sadness,1030,4,negative,-53.905170,-43.386887,-37.086166,-33.871140,-30.652399,-25.967619,...,-61.185417,-61.963745,-62.287804,-63.423450,-63.708729,-63.579025,-64.373978,-65.627625,-70.394920,-78.519653
12822,sadness,1036,1,negative,-31.754524,-31.717495,-29.293173,-27.252337,-24.792559,-24.326172,...,-59.602276,-59.736683,-59.998604,-60.810440,-61.052074,-61.299347,-62.677368,-63.914894,-68.689537,-77.104012
12823,sadness,1036,3,negative,-25.843838,-25.938826,-24.345428,-21.947716,-17.645626,-18.174150,...,-56.014198,-56.561848,-56.946644,-58.137333,-58.426796,-58.635120,-60.210060,-61.658447,-66.239624,-76.923187
12824,sadness,1036,8,negative,-23.881714,-24.504099,-24.566311,-22.493364,-19.697737,-20.646078,...,-50.514225,-50.690666,-50.787540,-52.253510,-52.281872,-52.611309,-53.727940,-54.972492,-59.620949,-71.365517


In [4]:
df['Emotion'].value_counts()

neutral     4709
joy         1743
sadness     1366
disgust     1355
fear        1340
surprise    1204
anger       1109
Name: Emotion, dtype: int64

In [5]:
anger_df = df[df['Emotion']=='anger']
df = df.append(anger_df.sample(4))

In [5]:
features = df.drop(['Emotion','Sentiment', 'Dialogue_ID', 'Utterance_ID'], axis = 1)

In [6]:
y = np.array(df['Emotion'].tolist())
y

array(['neutral', 'neutral', 'neutral', ..., 'sadness', 'sadness',
       'sadness'], dtype='<U8')

In [7]:
X = np.array(features)
X

array([[-40.78555679, -31.90338326, -28.50905418, ..., -61.93150711,
        -67.36791992, -75.37733459],
       [-46.09111786, -41.26459122, -34.1730957 , ..., -65.91607666,
        -69.98584747, -76.36937714],
       [-49.23120499, -42.71095276, -39.3041153 , ..., -69.56027222,
        -74.17944336, -79.33434296],
       ...,
       [-25.84383774, -25.93882561, -24.34542847, ..., -61.65844727,
        -66.23962402, -76.92318726],
       [-23.88171387, -24.50409889, -24.56631088, ..., -54.97249222,
        -59.62094879, -71.36551666],
       [-26.70499229, -23.45689011, -22.4702282 , ..., -62.21467972,
        -66.87866211, -77.40598297]])

In [8]:
lb = LabelEncoder()
label_y = to_categorical(lb.fit_transform(y))
label_y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [9]:
lb.classes_

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype='<U8')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, label_y, test_size = 0.2, random_state = 0)

In [11]:
y_train.shape

(10260, 7)

In [12]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [13]:
from tensorflow.keras import layers

In [15]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))

model.add(Conv1D(256, 8, padding='same', activation="relu"))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))

model.add(Conv1D(128, 8, padding='same', activation="relu"))
model.add(Conv1D(128, 8, padding='same', activation="relu"))

model.add(Conv1D(128, 8, padding='same', activation="relu"))

model.add(Conv1D(128, 8, padding='same', activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(8)))

model.add(Conv1D(64, 8, padding='same', activation="relu"))

model.add(Conv1D(64, 8, padding='same', activation="relu"))

    #model.add(Flatten())

model.add(layers.LSTM(256))

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(7)) 
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 128, 256)          2304      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 128, 256)          524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 256)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 16, 256)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 16, 128)           262272    
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 16, 128)           131200    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 16, 128)          

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
score = model.evaluate(X_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

81/81 [==============================] - 1s 18ms/step - loss: 1.9411 - accuracy: 0.1083
Pre-training accuracy: 10.8340%


In [16]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 72
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saved_model_LSTM.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/72
41/41 [==============================] - ETA: 0s - loss: 1.8002 - accuracy: 0.3645
Epoch 00001: val_loss improved from inf to 1.89924, saving model to saved_model_LSTM.hdf5
41/41 [==============================] - 29s 701ms/step - loss: 1.8002 - accuracy: 0.3645 - val_loss: 1.8992 - val_accuracy: 0.3628
Epoch 2/72
41/41 [==============================] - ETA: 0s - loss: 1.7892 - accuracy: 0.3682
Epoch 00002: val_loss improved from 1.89924 to 1.81254, saving model to saved_model_LSTM.hdf5
41/41 [==============================] - 28s 687ms/step - loss: 1.7892 - accuracy: 0.3682 - val_loss: 1.8125 - val_accuracy: 0.3628
Epoch 3/72
41/41 [==============================] - ETA: 0s - loss: 1.7853 - accuracy: 0.3682
Epoch 00003: val_loss improved from 1.81254 to 1.79950, saving model to saved_model_LSTM.hdf5
41/41 [==============================] - 28s 677ms/step - loss: 1.7853 - accuracy: 0.3682 - val_loss: 1.7995 - val_accuracy: 0.3628
Epoch 4/72
41/41 [==========================

Epoch 58/72
41/41 [==============================] - ETA: 0s - loss: 1.5254 - accuracy: 0.4321
Epoch 00058: val_loss did not improve from 1.69905
41/41 [==============================] - 27s 666ms/step - loss: 1.5254 - accuracy: 0.4321 - val_loss: 1.8953 - val_accuracy: 0.3745
Epoch 59/72
41/41 [==============================] - ETA: 0s - loss: 1.5084 - accuracy: 0.4343
Epoch 00059: val_loss did not improve from 1.69905
41/41 [==============================] - 27s 659ms/step - loss: 1.5084 - accuracy: 0.4343 - val_loss: 2.2185 - val_accuracy: 0.3523
Epoch 60/72
41/41 [==============================] - ETA: 0s - loss: 1.4995 - accuracy: 0.4369
Epoch 00060: val_loss improved from 1.69905 to 1.63619, saving model to saved_model_LSTM.hdf5
41/41 [==============================] - 27s 661ms/step - loss: 1.4995 - accuracy: 0.4369 - val_loss: 1.6362 - val_accuracy: 0.4053
Epoch 61/72
41/41 [==============================] - ETA: 0s - loss: 1.4613 - accuracy: 0.4518
Epoch 00061: val_loss did no